# COURSE: Master Python for scientific programming by solving projects
## PROJECT: Interpolation and extrapolation
#### TEACHER: Mike X Cohen, sincxpress.com
##### COURSE URL: udemy.com/course/maspy_x/?couponCode=202201

In [ ]:
# import all necessary modules
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal # for resample
from scipy import interpolate

# Down/upsample a time series

In [ ]:
# generate a smooth time series

N = 1000 # time points
k = 11 # k-point mean-smoothing kernel
timevec = np.arange(N)/N

# create smoothed time series
tsO = np.cumsum( np.random.randn(N) )
ts  = np.convolve(tsO,np.ones(k)/k,mode='same')

# plot them
plt.plot(timevec,tsO)
plt.plot(timevec,ts)
plt.xlim(timevec[[0,-1]])
plt.xlabel('Time (norm.)')
plt.show()

In [ ]:
# downsample to 1/2 the original rate

ts_ds = signal.resample(ts,int(N/2))
timevec_ds = signal.resample(timevec,int(N/2))

plt.plot(timevec,ts,'r')
plt.plot(timevec_ds,ts_ds,'b.')
# plt.xlim([.5,.56])
plt.show()

In [ ]:
# upsample to 3x the original rate

ts_us = signal.resample(ts,int(N*3))
timevec_us = signal.resample(timevec,int(N*3))

plt.plot(timevec,ts,'rs')
plt.plot(timevec_us,ts_us,'b.')
# plt.xlim([.6,.605])
plt.show()

# 1D interpolation

In [ ]:
# Create signal and visualize
sig = np.array([ 1,3,-2,2,9,10 ])

plt.plot(sig,'o',markersize=14)
plt.show()

In [ ]:
# using numpy

interp_factor = 2

orig_points = np.arange(len(sig))
interp_points = np.linspace(0,len(sig)-1,int(len(sig)*interp_factor))

interp_sig = np.interp(interp_points,orig_points,sig)

plt.plot(interp_points,interp_sig,'rs-')
plt.plot(orig_points,sig,'o',markersize=14)
plt.show()

In [ ]:
# using scipy

interp_factor = 2

interp_funL = interpolate.interp1d(orig_points,sig,bounds_error=False,kind='linear')
interp_funC = interpolate.interp1d(orig_points,sig,bounds_error=False,kind='cubic')
interp_sigL = interp_funL(interp_points)
interp_sigC = interp_funC(interp_points)

plt.plot(interp_points,interp_sigL,'rs-',label='linear')
plt.plot(interp_points,interp_sigC,'mp-',label='cubic')
plt.plot(orig_points,sig,'o',markersize=14,label='original')
plt.legend()
plt.show()

# 1D extrapolation

In [ ]:
# extrapolation

interp_factor = 2

orig_points = np.arange(len(sig))
interp_points = np.linspace(0,(len(sig)-1)*2,int(len(sig)*interp_factor))

interp_funL = interpolate.interp1d(orig_points,sig,bounds_error=False,kind='linear',fill_value='extrapolate')
interp_funC = interpolate.interp1d(orig_points,sig,bounds_error=False,kind='cubic',fill_value='extrapolate')
interp_sigL = interp_funL(interp_points)
interp_sigC = interp_funC(interp_points)

plt.plot(interp_points,interp_sigL,'rs-',label='linear')
plt.plot(interp_points,interp_sigC,'mp-',label='cubic')
plt.plot(orig_points,sig,'o',markersize=14,label='original')
plt.legend()
plt.ylim([-3,13])
plt.show()

# Resampling revisited


In [ ]:
# linear interpolation of upsampled signal

interp_funL = interpolate.interp1d(timevec_us,ts_us,bounds_error=False,kind='linear')
timevec_usRegular = np.linspace(timevec[0],timevec[-1],len(timevec_us))

ts_usRegular = interp_funL(timevec_usRegular)


plt.plot(timevec,ts,'rs',label='Original')
plt.plot(timevec_us,ts_us+2,'b.',label='Upsampled')
plt.plot(timevec_usRegular,ts_usRegular+4,'k.',label='Upsampled/interpolated')
plt.xlim([.6,.605])
plt.legend()
plt.show()

# Fix corrupted image with interpolation

In [ ]:
# create 2D function
x = np.linspace(-2*np.pi,2*np.pi,60)
X,Y = np.meshgrid(x,x)
Zgood = np.sin(np.log((X-2)**2 + np.abs((Y-3)**3)))

# visualize
plt.imshow(Zgood,extent=[x[0],x[-1],x[0],x[-1]],origin='lower')
plt.show()

In [ ]:
# select pixels to corrupt

prop_bad_pixels = .1

# total number of elements
num_elements = np.prod(Zgood.shape)

# random selection of pixels to trash
badpix_idx = np.random.rand(int(num_elements*prop_bad_pixels))*num_elements
badpix_idx = np.floor(badpix_idx).astype(int)
badpix_idx


In [ ]:
# destroy those pixels
Z = Zgood.copy()
Z[np.unravel_index(badpix_idx,Z.shape)] = np.nan

# or more explicitly:
badpix_i,badpix_j = np.unravel_index(badpix_idx,Z.shape)
Z[badpix_i,badpix_j] = np.nan


# visualize
plt.imshow(Z,extent=[x[0],x[-1],x[0],x[-1]],origin='lower')
plt.show()

In [ ]:
# show that .copy works as expected on numpy arrays
print(id(Z))
print(id(Zgood))

In [ ]:
# try with different variable types
l = 'asdf' # [1,2,'asdf']
ll = l.copy()

print(id(l))
print(id(ll))

In [ ]:
# find good and bad indices
badidx_i,badidx_j = np.where(np.isnan(Z))
gooidx_i,gooidx_j = np.where(np.isfinite(Z))

# interpolate using griddata
Znewpix = interpolate.griddata( (gooidx_i,gooidx_j),Z[np.isfinite(Z)],
                                (badidx_i,badidx_j) )

# replace the bad pixels
Zinterp = Z.copy()
Zinterp[badidx_i,badidx_j] = Znewpix

In [ ]:

# visualize
fig,ax = plt.subplots(1,3,figsize=(14,7))
ax[0].imshow(Zgood,extent=[x[0],x[-1],x[0],x[-1]],origin='lower')
ax[0].set_title('Original')

ax[1].imshow(Z,extent=[x[0],x[-1],x[0],x[-1]],origin='lower')
ax[1].set_title('Corrupted')

ax[2].imshow(Zinterp,extent=[x[0],x[-1],x[0],x[-1]],origin='lower')
ax[2].set_title('Interpolated')

plt.show()

In [ ]:
# show difference map
diffmap = np.abs(Zgood-Zinterp)

plt.imshow(diffmap,vmin=0,vmax=np.max(Zgood))
plt.show()

# Bonus: Draw a Necker cube

In [ ]:
from matplotlib.patches import Rectangle

# offset for "upper" square
o = 1/3

# the rectangles
r1 = Rectangle((0,0),1,1,facecolor='none',edgecolor='k',linewidth=3)
r2 = Rectangle((o,o),1,1,facecolor='none',edgecolor='k',linewidth=3)

fig, ax = plt.subplots(figsize=(5,5))
ax.add_patch(r1)
ax.add_patch(r2)
ax.set_xlim([-.5,1.5])
ax.set_ylim([-.5,1.5])


# lines connecting the two squares
plt.plot([0,o],[0,o],'k',linewidth=3)
plt.plot([0,o],[1,1+o],'k',linewidth=3)
plt.plot([1,1+o],[0,o],'k',linewidth=3)
plt.plot([1,1+o],[1,1+o],'k',linewidth=3)

# the dots on the edges (the long way)
# plt.plot(0,0,'ko',markersize=15,markerfacecolor='w')
# plt.plot(0,1,'ko',markersize=15,markerfacecolor='w')
# plt.plot(1,0,'ko',markersize=15,markerfacecolor='w')
# plt.plot(1,1,'ko',markersize=15,markerfacecolor='w')
# plt.plot(o,o,'ko',markersize=15,markerfacecolor='w')
# plt.plot(o,1+o,'ko',markersize=15,markerfacecolor='w')
# plt.plot(1+o,o,'ko',markersize=15,markerfacecolor='w')
# plt.plot(1+o,1+o,'ko',markersize=15,markerfacecolor='w')

# the more efficient way
edges = [ [0,0],[0,1],[1,0],[1,1],[o,o],[o,1+o],[1+o,o],[1+o,1+o] ]
for i in edges:
  plt.plot(i[0],i[1],'ko',markersize=15,markerfacecolor='w')

plt.axis('off')
plt.show()